In [1]:
url = 'https://mdk-arbat.ru/catalog/?subj_id=3217'

In [ ]:
import urllib, requests, socket, re, lxml, io, bs4, sqlite3, pandas, sqlalchemy
from bs4 import BeautifulSoup
from requests.compat import urljoin, quote_plus, urlparse, unquote

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
    'Accept-Language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Referer': 'https://mdk-arbat.ru/catalog?subj_id=47',
    'Connection': 'keep-alive',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
}
html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, 'lxml')

In [3]:
book_links=[]
book_prices=[]
book_genres=[]
book_names=[]
book_authors=[]
book_publishers=[]



for genre_tag in soup.find_all('ul', class_='tg-bookscategories'):
    genre = genre_tag.find('a').text
    book_genres.append(genre)

for book_tag in soup.find_all('div', class_='tg-booktitle'):
    book_title = book_tag.find('a').text
    book_link = urljoin(url, book_tag.find('a')['href'])
    book_names.append(book_title)
    book_links.append(book_link)

for book_author in soup.find_all('span', class_='tg-bookwriter'):
  author = book_author.find('a').text
  book_authors.append(author)

for book_price in soup.find_all('span', class_='tg-bookprice'):
  price = book_price.find('ins').text[:-1].strip()
  book_prices.append(price)

for link in book_links:
  html_book = requests.get(link, headers=headers).text
  soup_book = BeautifulSoup(html_book, 'lxml')
  publisher = soup_book.find('meta', itemprop='publisher')['content']
  book_publishers.append(publisher)

# и тут я понял, что все странно парсится - книги совершенно не те, которые по ссылке. Поэтому погнали ботать селениум